In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-pre

In [2]:
# suppress warnings 
import warnings 
warnings.simplefilter(action='ignore')

In [3]:
# Data cleaning packages.
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer

In [4]:
# Data Read
train_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")
train_protein = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
train_peptide = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
clinical_sup = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [5]:
# Trend Only - responses + vist_month
# Both data with and without protein info
train = pd.concat([train_clinical, clinical_sup])
print(train.head())
print(train.tail())

  visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0     55_0          55            0     10.0      6.0     15.0      NaN   
1     55_3          55            3     10.0      7.0     25.0      NaN   
2     55_6          55            6      8.0     10.0     34.0      NaN   
3     55_9          55            9      8.0      9.0     30.0      0.0   
4    55_12          55           12     10.0     10.0     41.0      0.0   

  upd23b_clinical_state_on_medication  
0                                 NaN  
1                                 NaN  
2                                 NaN  
3                                  On  
4                                  On  
      visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
2218   65382_0       65382            0      NaN      NaN      0.0      NaN   
2219   65405_0       65405            0      5.0     16.0     31.0      0.0   
2220   65405_5       65405            5      NaN      NaN     57.0      

In [6]:
# Model Imports
#from sklearn.ensemble import RandomForestRegressor

#from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [7]:
# Tuning Grid 
xgb_grid = {'n_estimators': [100, 200],
            'max_depth': [3, 10],
            'min_samples_split': [2],
            'learning_rate': [0.01, 0.015],
            'loss': ['absolute_error']}

In [8]:
# Empty dictionary 
models = {}
# Response variable names
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

# Loop to train a model for each of response
for u in target: 
    # Drop NA values in current response target 
    temp = train.dropna(subset = [u])
    
    # visit_month as sole predictor
    X = temp['visit_month']
    y = temp[u]
    
    # Model
    xgb = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = xgb_grid, 
        cv = 2, n_jobs = 30, verbose = 2)
    
    trained = xgb.fit(X.values.reshape(-1, 1), y)
    
    models[u] = trained.best_estimator_


Fitting 2 folds for each of 8 candidates, totalling 16 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Fitting 2 folds for each of 8 candidates, totalling 16 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Fitting 2 folds for each of 8 candidates, totalling 16 fits
Fitting 2 folds for each of 8 candidates, totalling 16 fits


In [9]:
# Prediction and Formatting Function 
def get_prediction(train, models):
    
    # Fill NAs
    train = train.fillna(0)
    
    for u in target:
        # Add results column 
        train['result_' + str(u)] = 0
        
        # Extract predictor
        X = train['visit_month']
        
        # Make predictions 
        train['result_' + str(u)] = models[u].predict(X.values.reshape(-1, 1))
        
    # Formatting 
    
    # Empty DF to store results. 
    result = pd.DataFrame()
    
    # Loop over months 
    for m in [0, 6, 12, 24]:
        # Loop over responses 
        for u in [1, 2, 3, 4]:

            temp = train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]

            result = pd.concat([result, temp])            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result
    

In [10]:
train_preds = get_prediction(train, models)
train_preds

,prediction_id,rating
0,55_0_updrs_1_plus_0_months,5.000000
1,55_3_updrs_1_plus_0_months,4.732303
2,55_6_updrs_1_plus_0_months,5.000000
3,55_9_updrs_1_plus_0_months,5.000000
4,55_12_updrs_1_plus_0_months,5.000000
...,...,...
2218,65382_0_updrs_4_plus_24_months,0.000000
2219,65405_0_updrs_4_plus_24_months,0.000000
2220,65405_5_updrs_4_plus_24_months,0.000000
2221,65530_0_updrs_4_plus_24_months,0.000000


In [11]:
import amp_pd_peptide_310
amp_pd_peptide_310.make_env.func_dict['__called__'] = False
env = amp_pd_peptide_310.make_env()
iter_test = env.iter_test()

In [12]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    
    result = get_prediction(test, models)
    
    print(result.head())
    
    env.predict(result)
    

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
                   prediction_id     rating
0   3342_0_updrs_1_plus_0_months   5.000000
4  50423_0_updrs_1_plus_0_months   5.000000
0   3342_0_updrs_2_plus_0_months   4.668972
4  50423_0_updrs_2_plus_0_months   4.668972
0   3342_0_updrs_3_plus_0_months  18.746929
                   prediction_id     rating
0   3342_6_updrs_1_plus_0_months   5.000000
4  50423_6_updrs_1_plus_0_months   5.000000
0   3342_6_updrs_2_plus_0_months   5.331028
4  50423_6_updrs_2_plus_0_months   5.331028
0   3342_6_updrs_3_plus_0_months  20.000000
